In [1]:
!pip install google-cloud-bigquery

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

In [3]:
#q1

bq = bigquery.Client()

no_cache = bigquery.QueryJobConfig(use_query_cache=False)

query_q1 = """
SELECT geo_id
FROM `bigquery-public-data.geo_us_boundaries.counties`
WHERE county_name = 'Dane'
"""

dane = bq.query(query_q1, job_config=no_cache)
df = dane.to_dataframe()
geo_id_value = df.loc[0, "geo_id"]
geo_id_value

'55025'

In [4]:
#q2

query_q2 = """
SELECT state_fips_code, COUNT(*) AS county_count
FROM `bigquery-public-data.geo_us_boundaries.counties`
GROUP BY state_fips_code
ORDER BY county_count DESC
LIMIT 5
"""

fips_5 = bq.query(query_q2, job_config=no_cache)
df = fips_5.to_dataframe()
df["county_count"] = df["county_count"].astype(int)
top5 = dict(zip(df["state_fips_code"], df["county_count"]))

top5

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [5]:
#q3
from google.cloud import bigquery
import math


bytes1 = dane.total_bytes_billed / (1024**2)
bytes2 = fips_5.total_bytes_billed / (1024**2)

query_costs = { 
    "q1": f"{round(bytes1)} MB",
    "q2": f"{round(bytes2)} MB"
}



query_costs

{'q1': '10 MB', 'q2': '10 MB'}

In [6]:
# Create Dataset:
dataset_id = "cs544-fa24.p8"

dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"

dataset = client.create_dataset(dataset, exists_ok=True)
dataset.dataset_id

'p8'

In [7]:
#q4
datasets = list(client.list_datasets())
dataset_ids = [dataset.dataset_id for dataset in datasets]
dataset_ids

['p8']

In [8]:
from google.cloud import bigquery
#check table
client = bigquery.Client()
tables = client.list_tables("cs544-fa24.p8")

print("Tables in dataset:")
for table in tables:
    table.table_id


Tables in dataset:


In [9]:
client.query("SELECT * FROM `cs544-fa24.p8.hdma` LIMIT 1").result()
client.query("SELECT * FROM `bigquery-public-data.geo_us_boundaries.counties` LIMIT 1").result()

In [10]:
#q5
from google.cloud import bigquery

client = bigquery.Client()

query = """
SELECT counties.county_name AS county_name, COUNT(*) AS application_count
FROM `cs544-fa24.p8.hdma` hdma
JOIN `bigquery-public-data.geo_us_boundaries.counties` counties
ON hdma.county_code = SUBSTR(CONCAT(counties.state_fips_code, counties.county_fips_code), -5) -- Match last 5 characters
WHERE hdma.state_code = 'WI' -- Filter for Wisconsin
GROUP BY county_name
ORDER BY application_count DESC
LIMIT 10
"""

result = client.query(query).result()

applications_per_county = {row.county_name: row.application_count for row in result}

applications_per_county

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [11]:

from google.cloud import bigquery
#link the sheet
#ChatGPT: Link this google sheet [url] to the BigQuery API...
dataset_id = "cs544-fa24.p8"
table_id = "applications"
sheet_url = "https://docs.google.com/spreadsheets/d/13e14LzDDm9U4y2KddlKFAy7exNdbo1OwJa-OTe4ywiw/edit?gid=1070618599"

schema = [
    bigquery.SchemaField("application_id", "STRING"),
    bigquery.SchemaField("income", "FLOAT"),
    bigquery.SchemaField("loan_amount", "FLOAT"),
    bigquery.SchemaField("latitude", "FLOAT"),
    bigquery.SchemaField("longitude", "FLOAT"),
]

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [sheet_url]
external_config.options.skip_leading_rows = 1
external_config.schema = schema

table = bigquery.Table(f"{dataset_id}.{table_id}")
table.external_data_configuration = external_config

table = client.create_table(table, exists_ok=True)
print(f"Linked Google Sheet to BigQuery table: {table.table_id}")

Linked Google Sheet to BigQuery table: applications


In [12]:
#q6
chosen_income = 10000000

query = f"""
SELECT COUNT(*) AS application_count
FROM `cs544-fa24.p8.applications`
WHERE income = {chosen_income}
"""

result = client.query(query).result()
application_count = [row.application_count for row in result][0]
application_count

2

In [13]:
#q7
query_create_model = """
CREATE OR REPLACE MODEL `p8.my_linear_model`
OPTIONS(model_type='linear_reg') AS
SELECT
  income AS x,
  loan_amount AS label
FROM `p8.applications`
WHERE income IS NOT NULL AND loan_amount IS NOT NULL

"""

client.query(query_create_model).result()

query_evaluate = """
SELECT r2_score
FROM ML.EVALUATE(MODEL `p8.my_linear_model`)
"""

eval_df = client.query(query_evaluate).to_dataframe()
r2_score = float(eval_df.loc[0, 'r2_score'])

r2_score

0.03905723794687299

In [14]:
#q8
query_q8 = """
SELECT MIN(ST_DISTANCE(
    ST_GEOGPOINT(longitude, latitude),
    ST_GEOGPOINT(-89.3842, 43.0747)
  )) AS min_distance
FROM `p8.applications`
"""

df_q8 = client.query(query_q8).to_dataframe()
min_distance = float(df_q8.loc[0, 'min_distance'])
min_distance

0.0

In [15]:
#q9
query_q9 = """
SELECT c.county_name, COUNT(*) AS count
FROM `p8.applications` a
JOIN `bigquery-public-data.geo_us_boundaries.counties` c
  ON c.state_fips_code = '55' 
  AND ST_WITHIN(
    ST_GEOGPOINT(a.longitude, a.latitude),
    c.county_geom
  )
GROUP BY c.county_name
ORDER BY count DESC
"""

df_q9 = client.query(query_q9).to_dataframe()
county_dict = {row['county_name']: int(row['count']) for _, row in df_q9.iterrows()}
county_dict

{'Dane': 71,
 'Door': 9,
 'Milwaukee': 7,
 'Jefferson': 4,
 'Bayfield': 3,
 'Brown': 3,
 'Marinette': 3,
 'Oneida': 3,
 'Sauk': 3,
 'Juneau': 2,
 'Shawano': 2,
 'Fond du Lac': 2,
 'Price': 2,
 'St. Croix': 2,
 'Waushara': 1,
 'Ashland': 1,
 'Winnebago': 1,
 'Sawyer': 1,
 'Columbia': 1,
 'Forest': 1,
 'Taylor': 1,
 'Barron': 1,
 'Wood': 1,
 'Oconto': 1,
 'Douglas': 1,
 'Waukesha': 1,
 'Vilas': 1,
 'Iowa': 1,
 'Walworth': 1,
 'Green': 1,
 'Rock': 1}

In [16]:
#q10
query_q10 = """
WITH dane AS (
  SELECT county_geom
  FROM `bigquery-public-data.geo_us_boundaries.counties`
  WHERE county_name = 'Dane' AND state_fips_code='55'
)
SELECT c.county_name
FROM `bigquery-public-data.geo_us_boundaries.counties` c, dane
WHERE c.state_fips_code='55'
  AND c.county_name != 'Dane'
  AND ST_TOUCHES(c.county_geom, dane.county_geom)
ORDER BY county_name
"""

df_q10 = client.query(query_q10).to_dataframe()
bordering_counties = df_q10['county_name'].tolist()
bordering_counties

['Columbia', 'Dodge', 'Green', 'Iowa', 'Jefferson', 'Rock', 'Sauk']